<img src=docs/tudelft_logo.jpg width=50%>

## Data-driven Design and Analyses of Structures and Materials (3dasm)

## Lecture 5

### Miguel A. Bessa | <a href = "mailto: M.A.Bessa@tudelft.nl">M.A.Bessa@tudelft.nl</a>  | Associate Professor

**What:** A lecture of the "3dasm" course

**Where:** This notebook comes from this [repository](https://github.com/bessagroup/3dasm_course)

**Reference for entire course:** Murphy, Kevin P. *Probabilistic machine learning: an introduction*. MIT press, 2022. Available online [here](https://probml.github.io/pml-book/book1.html)

**How:** We try to follow Murphy's book closely, but the sequence of Chapters and Sections is different. The intention is to use notebooks as an introduction to the topic and Murphy's book as a resource.
* If working offline: Go through this notebook and read the book.
* If attending class in person: listen to me (!) but also go through the notebook in your laptop at the same time. Read the book.
* If attending lectures remotely: listen to me (!) via Zoom and (ideally) use two screens where you have the notebook open in 1 screen and you see the lectures on the other. Read the book.

**Optional reference (the "bible" by the "bishop"... pun intended 😆) :** Bishop, Christopher M. *Pattern recognition and machine learning*. Springer Verlag, 2006.

**References/resources to create this notebook:**
* [Car figure](https://korkortonline.se/en/theory/reaction-braking-stopping/)

Apologies in advance if I missed some reference used in this notebook. Please contact me if that is the case, and I will gladly include it here.

## **OPTION 1**. Run this notebook **locally in your computer**:
1. Confirm that you have the 3dasm conda environment (see Lecture 1).

2. Go to the 3dasm_course folder in your computer and pull the last updates of the [repository](https://github.com/bessagroup/3dasm_course):
```
git pull
```
3. Open command window and load jupyter notebook (it will open in your internet browser):
```
conda activate 3dasm
jupyter notebook
```
4. Open notebook of this Lecture.

## **OPTION 2**. Use **Google's Colab** (no installation required, but times out if idle):

1. go to https://colab.research.google.com
2. login
3. File > Open notebook
4. click on Github (no need to login or authorize anything)
5. paste the git link: https://github.com/bessagroup/3dasm_course
6. click search and then click on the notebook for this Lecture.

In [1]:
# Basic plotting tools needed in Python.

import matplotlib.pyplot as plt # import plotting tools to create figures
import numpy as np # import numpy to handle a lot of things!
from IPython.display import display, Math # to print with Latex math

%config InlineBackend.figure_format = "retina" # render higher resolution images in the notebook
plt.style.use("seaborn") # style for plotting that comes from seaborn
plt.rcParams["figure.figsize"] = (8,4) # rescale figure size appropriately for slides

## Outline for today

* Bayesian inference for one hidden rv
    - Prior
    - Likelihood
    - Marginal likelihood
    - Posterior
    - Gaussian pdf's product

**Reading material**: This notebook + Chapter 3

In [2]:
# This cell is hidden during presentation. It's just to define a function to plot the governing model of
# the car stopping distance problem. Defining a function that creates a plot allows to repeatedly run
# this function on cells used in this notebook.
def car_fig(ax):
    real_x = np.linspace(3, 83, 1000)
    real_mu_y = 1.5*real_x + 0.1*(real_x**2) # Recall: E[z*x+0.1*x^2] = E[z]*E[x]+0.1*E[x^2]
    real_sigma_y = np.sqrt( 0.5**2*real_x**2 ) # Recall: V[z*x+x^2] = V[z*x]+V[x^2]=...=sigma_z^2+mu_x^2
    ax.set_xlabel("x (m/s)", fontsize=20) # create x-axis label with font size 20
    ax.set_ylabel("y (m)", fontsize=20) # create y-axis label with font size 20
    ax.set_title("Car stopping distance problem", fontsize=20); # create title with font size 20
    ax.plot(real_x, real_mu_y, 'k:', label="Governing model $\mu_y$")
    ax.fill_between(real_x, real_mu_y - 1.9600 * real_sigma_y,
                    real_mu_y + 1.9600 * real_sigma_y,
                    color='k', alpha=0.2,
                    label='95% confidence interval ($\mu_y \pm 1.96\sigma_y$)') # plot 95% credence interval
    ax.legend(fontsize=15)

### A slightly more complicated car stopping distance problem

<img src="docs/reaction-braking-stopping.svg" title="Car stopping distance" width="25%" align="right">

Today we will finally do some predictions!

Recall the Homework of Lecture 4, and consider the car stopping distance problem for constant velocity $x=75$ m/s and for which **it is known** that $z_2 \sim \mathcal{N}(z_2|\mu_{z_2}=0.1,\sigma_{z_2}^2=0.01^2)$.

The only information that we do not know is the driver's reaction time $z$ (here we are calling $z$ instead of $z_1$ as in Lecture 4 because this is the only hidden variable).

* Can we predict $p(y)$ without knowing $p(z)$?

Yes!! If we use Bayes' rule!

### Recall the Homework of Lecture 4

<img src="docs/reaction-braking-stopping.svg" title="Car stopping distance" width="25%" align="right">

From last lecture's Homework, you demonstrated that the conditional pdf of the stopping distance given the reaction time $z$ (for convenience we write here $z$ instead of $z_1$) is

$$
p(y|z) = \mathcal{N}\left(y | \mu_{y|z}=w z+b, \sigma_{y|z}^2=s^2\right)
$$

where $w$, $b$ and $s$ are all constants that you determined to be:

$w=x=75$

$b=x^2\mu_{z_2}=75^2\cdot0.1=562.5$

$s^2=(x^2 \sigma_{z_2})^2=(75^2\cdot0.01)^2=56.25^2$

because we are considering the car is going at constant velocity $x=75$ m/s and that we know $z_2= \mathcal{N}(z_2|\mu_{z_2}=0.1,\sigma_{z_2}^2=0.01^2)$.

### Understanding the Bayes' rule

$\require{color}$
$$
{\color{green}p(z|y)} = \frac{ {\color{blue}p(y|z)}{\color{red}p(z)} } {p(y)}
$$

* ${\color{red}p(z)}$ is the **prior distribution**
* ${\color{blue}p(y)}$ is the **observation distribution** (conditional pdf)
* $p(y)$ is the **marginal distribution**
* ${\color{green}p(z|y)}$ is the **posterior distribution**

### A note about the term "distribution"

The term distribution can mean two things:
1. For **continuous** <a title="random variables">rv's</a>, the term *distribution* means *probability density function* (<a title="probability density function">pdf</a>).

2. For **discrete** <a title="random variables">rv's</a> the term *distribution* means *probability mass function* (<a title="probability mass function">pmf</a>), as we will see later in the course.

We won't talk about categorical distributions or <a title="probability mass functions">pmf's</a> for a while. So, for now, when you see the term *distribution* it is the same as saying <a title="probability density function">pdf</a>.

### Understanding the Bayes' rule

Let's start by understanding the usefulness of Bayes' rule by calculating the posterior $p(z|y)$ for the car stopping distance problem (Homework of Lecture 4).

As we mentioned, for our problem we know the **observation distribution**:

$p(y|z) = \mathcal{N}\left(y | \mu_{y|z}=w z+b, \sigma_{y|z}^2\right)$

where $\sigma_{y|z} = \text{const}$, as well as $w$ and $b$. 

but we **don't know** the prior $p(z)$.

### Prior: our beliefs about the problem

If we have absolutely no clue about what the distribution of the hidden rv $z$ is, then we can use a **Uniform distribution** (a.k.a. uninformative prior).

This distribution assigns equal probability to any value of $z$ within an interval $z \in (z_{min}, z_{max})$.

$$
p(z) = \frac{1}{C_z}
$$

where $C_z = z_{max}-z_{min}$ is the **normalization constant** of the Uniform pdf, i.e. the value that guarantees that $p(z)$ integrates to one.

As we will see, the actual values of $z_{max}$ and $z_{min}$ won't make any difference. But if you don't to use letters for constants, you can consider that $z_{min} = 0$ seconds is the limit of the fastest reaction time that is humanly possible, and $z_{max} = 3$ seconds is the slowest reaction time of a human being (you could get this additional information by asking a doctor).

### Summary of our Model

1. The **observation distribution**:

$$\begin{align}
p(y|z) &= \mathcal{N}\left(y | \mu_{y|z}=w z+b, \sigma_{y|z}^2\right) \\
&= \frac{1}{C_{y|z}} \exp\left[ -\frac{1}{2\sigma_{y|z}^2}(y-\mu_{y|z})^2\right]
\end{align}
$$

where $C_{y|z} = \sqrt{2\pi \sigma_{y|z}^2}$ is the **normalization constant** of the Gaussian pdf, and where $\mu_{y|z}=w z+b$, with $w$, $b$ and $\sigma_{y|z}^2$ being constants, as previously mentioned.

2. and the **prior distribution**: $p(z) = \frac{1}{C_z}$

where $C_z = z_{max}-z_{min}$ is the **normalization constant** of the Uniform pdf, i.e. the value that guarantees that $p(z)$ integrates to one.

### Posterior from Bayes' rule

Since we have defined the **observation distribution** and the **prior distribution**, we can now compute the posterior distribution from Bayes' rule.

But this requires a bit of algebra... Let's do it!

First, in order to apply Bayes' rule $p(z|y) = \frac{ p(y|z)p(z)}{p(y)}$ we need to calculate $p(y)$.

$p(y)$ is obtained by marginalizing the joint distribution wrt $z$:

$
p(y) = \int p(y|z)p(z) dz
$

which implies an integration over $z$. So, let's rewrite $p(y|z)$ so that the integration becomes easier.

$$\begin{align}
p(y|z) &= \mathcal{N}\left(y | \mu_{y|z}=w z+b, \sigma_{y|z}^2\right) \\
&= \frac{1}{C_{y|z}} \exp\left[ -\frac{1}{2\sigma_{y|z}^2}(y-(w z+b))^2\right] \\
&= \frac{1}{C_{y|z}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\} \\
&= \frac{1}{|w|}\frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\}
\end{align}
$$


Note: This Gaussian pdf $\mathcal{N}\left(z | \frac{y-b}{w}, \left(\frac{\sigma_{y|z}}{w}\right)^2\right)$ is unnormalized when written wrt $z$ (due to $\frac{1}{|w|}$).

We can now calculate the marginal distribution $p(y)$:

$$
\begin{align}
p(y) &= \int p(y|z)p(z) dz \\
&= \int \frac{1}{|w|}\frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\} \frac{1}{C_z} dz
\end{align}
$$

We can rewrite this expression as,

$$\require{color}
\begin{align}
p(y) &= \frac{1}{|w|\cdot C_z} {\color{blue}\int \frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\} dz} \\
\end{align}
$$

What is the result for the <font color='blue'>blue term</font>?

From where we conclude that the marginal distribution is:

$$\require{color}
p(y) = \frac{1}{|w| C_z }
$$

So, now we can determine the posterior:

$$\require{color}
\begin{align}
p(z|y) &= \frac{ p(y|z)p(z)}{p(y)} \\
&= |w| C_z \cdot \frac{1}{|w|}\frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\} \cdot \frac{1}{C_z}\\
&= \frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}}\exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y-b}{w}\right)^2\right]\right\}
\end{align}
$$

which is a **normalized** Gaussian pdf in $z$: $\mathcal{N}\left(z | \frac{y-b}{w}, \left(\frac{\sigma_{y|z}}{w}\right)^2\right)$

* **This is what the Bayes' rule does!** Computes the posterior $p(z|y)$ from $p(y|z)$ and $p(z)$.

## Why should we care about the Bayes' rule?

There are a few reasons:

1. As we will see, models are usually (always?) wrong.


2. But our beliefs may be a bit closer to reality! Bayes' rule enables us to get better models if our beliefs are reasonable!


3. We don't observe distributions. We observe **DATA**. Bayes' rule is a very powerful way to predict the distribution of our quantity of interest (here: $y$) from data!

## Bayes' rule applied to observed data

Previously, we already introduced Bayes' rule when applied to observed data $\mathcal{D}_y$.

$\require{color}$
$$
{\color{green}p(z|y=\mathcal{D}_y)} = \frac{ {\color{blue}p(y=\mathcal{D}_y|z)}{\color{red}p(z)} } {p(y=\mathcal{D}_y)} = \frac{ {\color{magenta}p(y=\mathcal{D}_y, z)} } {p(y=\mathcal{D}_y)}
$$

* ${\color{red}p(z)}$ is the **prior** distribution
* ${\color{blue}p(y=\mathcal{D}_y|z)}$ is the **likelihood** function
* ${\color{magenta}p(y=\mathcal{D}_y, z)}$ is the **joint likelihood** (product of likelihood function with prior distribution)
* $p(y=\mathcal{D}_y)$ is the **marginal likelihood**
* ${\color{green}p(z|y=\mathcal{D}_y)}$ is the **posterior**

We can write Bayes' rule as <font color='green'>posterior</font> $\propto$ <font color='blue'>likelihood</font> $\times$ <font color='red'>prior </font>, where we are ignoring the denominator $p(y=\mathcal{D}_y)$ because it is just a **constant** independent of the hidden variable $z$.

## Bayes' rule applied to observed data

But remember that Bayes' rule is just a way to calculate the posterior:

$$
p(z|y=\mathcal{D}_y) = \frac{ p(y=\mathcal{D}_y|z)p(z) } {p(y=\mathcal{D}_y)}
$$

Usually, what we really want is to be able to predict the distribution of the quantity of interest (here: $y$) after observing some data $\mathcal{D}_y$:

$$\require{color}
{\color{orange}p(y|y=\mathcal{D}_y)} = \int p(y|z) p(z|y=\mathcal{D}_y) dz
$$

which is often written in simpler notation: $p(y|\mathcal{D}_y) = \int p(y|z) p(z|\mathcal{D}_y) dz$

### Bayesian inference for car stopping distance problem

Now we will solve the first Bayesian ML problem from some given data $y=\mathcal{D}_y$:

| $y_i$ (m) |
| ---- |
| 601.5 |
| 705.9 |
| 693.8 |
| ...   |
| 711.3 |

where the data $\mathcal{D}_y$ could be a Pandas dataframe with $N$ data points ($N$ rows).

* **Very Important Question (VIQ)**: Can we calculate the **likelihood** function from this data?

### Likelihood for car stopping distance problem

Of course! As we saw a few cells ago, the **likelihood** is obtained by evaluating the **observation distribution** at the data $\mathcal{D}_y$.

Noting that each observation in $\mathcal{D}_y$ is independent of each other, then:

$$
p(y=\mathcal{D}_y | z) = \prod_{i=1}^{N} p(y=y_i|z) = p(y=y_1|z)p(y=y_2|z) \cdots p(y=y_N|z)
$$

which gives the **probability density** of observing that data if using our observation distribution (part of our model!).

#### Calculating the likelihood

Let's calculate it:

$$
\begin{align}
p(y=\mathcal{D}_y | z) &= \prod_{i=1}^{N} p(y=y_i|z) \\
&= \prod_{i=1}^{N} \frac{1}{C_{y|z}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y_i-b}{w}\right)^2\right]\right\}
\end{align}
$$

This seems a bit daunting... I know. Do not dispair yet!

##### Product of Gaussian pdf's of the same rv $z$

It can be shown that the product of $N$ univariate Gaussian pdf's of the same rv $z$ is:

$$
\prod_{i=1}^{N} \mathcal{N}(z|\mu_i, \sigma_i^2) = C \cdot \mathcal{N}(z|\mu, \sigma^2)
$$

with mean: $\mu = \sigma^2 \left( \sum_{i=1}^{N} \frac{\mu_i}{\sigma_i^2}\right)$

variance: $\sigma^2= \frac{1}{\sum_{i=1}^{N} \frac{1}{\sigma_i^2}}$

and normalization constant: $C = \frac{1}{2\pi^{(N-1)/2}}\sqrt{\frac{\sigma^2}{\prod_{i=1}^n \sigma_i^2}} \exp\left[-\frac{1}{2}\left(\sum_{i=1}^{N} \frac{\mu_i^2}{\sigma_i^2} - \frac{\mu^2}{\sigma^2}\right)\right]$

Curiosity: the normalization constant $C$ is itself a Gaussian! You can see it more clearly if you consider $N=2$

Note that the normalization constant shown in the previous cell can also be written as:

$$
C = \frac{1}{2\pi^{(N-1)/2}}\sqrt{\frac{\sigma^2}{\prod_{i=1}^n \sigma_i^2}} \exp\left[-\frac{1}{2}\left(\sum_{i=1}^{N-1}\sum_{j=i+1}^{N} \frac{(\mu_i-\mu_j)^2}{\sigma_i^2 \sigma_j^2}\sigma^2\right)\right]
$$

# <font color='red'>HOMEWORK</font>

Show that the product of two Gaussian <a title="probability density functions">pdf's</a> for the same <a title="random variable">rv</a> $z$ is:

$\mathcal{N}(z|\mu_1, \sigma_1^2)\cdot \mathcal{N}(z|\mu_2, \sigma_2^2)= C \cdot \mathcal{N}(z | \mu, \sigma^2)$

$$
\begin{align}
\sigma^2&=\frac{1}{\sigma_1^2+\sigma_2^2}\\
\mu&=\frac{1}{\sigma^2}(\frac{\mu_1}{\sigma_1^2} +  \frac{\mu_2}{\sigma_2^2})\\
C &= \frac{1}{\sqrt{2\pi(\sigma_1^2+\sigma_2^2)}} \exp\left[-\frac{1}{2(\sigma_1^2+\sigma_2^2)}(\mu_1-\mu_2)^2\right]
\end{align}
$$

#### Back to calculating the likelihood

$$
\begin{align}
p(y=\mathcal{D}_y | z) &= \prod_{i=1}^{N} p(y=y_i|z) \\
&= \prod_{i=1}^{N} \frac{1}{|w|} \frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y_i-b}{w}\right)^2\right]\right\} \\
&= \frac{1}{|w|^N} \prod_{i=1}^{N} \frac{1}{\sqrt{2\pi \left(\frac{\sigma_{y|z}}{w}\right)^2}} \exp\left\{ -\frac{1}{2\left(\frac{\sigma_{y|z}}{w}\right)^2}\left[z-\left(\frac{y_i-b}{w}\right)^2\right]\right\}
\end{align}
$$

So, using the result of a product of $N$ Gaussian pdf's to calculate the likelihood, and noting that $\sigma_i = \frac{\sigma_{y|z}}{w}$ and $\mu_i = \frac{y_i - b}{w}$ we get:

$$
p(y=\mathcal{D}_y | z) = \frac{1}{|w|^N} \cdot C \cdot \frac{1}{2\pi \sigma^2} \exp\left[ -\frac{1}{2\sigma^2}(z-\mu)^2\right]
$$

where

$\mu = \frac{\sigma^2}{\sigma_i^2} \sum_{i=1}^N \mu_i = \frac{w^2\sigma^2}{\sigma_{y|z}^2} \sum_{i=1}^N \mu_i$

$\sigma^2 = \frac{1}{ \sum_{i=1}^N \frac{1}{\sigma_i^2} } = \frac{1}{ \sum_{i=1}^N \frac{w^2 N}{\sigma_{y|z}^2} } = \frac{\sigma_{y|z}^2}{\sum_{i=1}^N w^2 N}$

$
C = \frac{1}{2\pi^{(N-1)/2}} \sqrt{\frac{\sigma^2}{\left( \frac{\sigma_{y|z}^2}{w^2}\right)^N}} \exp\left[-\frac{1}{2}\left(\frac{w^2}{\sigma_{y|z}^2}\sum_{i=1}^N \mu_i - \frac{\mu^2}{\sigma^2}\right) \right]
$

#### Calculating the marginal likelihood

$$\begin{align}
p(y=\mathcal{D}_y) &= \int p(y=\mathcal{D}_y | z) p(z) dz \\
&= \int \frac{1}{|w|^N} C \cdot \mathcal{N}(z|\mu, \sigma^2)\cdot \frac{1}{C_z} dz\\
&= \frac{C}{|w|^N C_z} \int \cdot \mathcal{N}(z|\mu, \sigma^2)\cdot \\
\end{align}
$$

We can now calculate the posterior:

$$\begin{align}
p(z|y=\mathcal{D}_y) &= \frac{ p(y=\mathcal{D}_y|z)p(z) } {p(y=\mathcal{D}_y)} \\
&= \frac{1}{p(y=\mathcal{D}_y)} \cdot \frac{1}{|w|^N} C \cdot \mathcal{N}(z|\mu,\sigma^2) \cdot \frac{1}{C_z} \\
&= \mathcal{N}(z|\mu, \sigma^2)
\end{align}
$$

#### Calculating the Predictive Posterior Distribution (PPD)

Having found the posterior, we can determine the PPD:

$$
p(y|\mathcal{D}_y) = \int p(y| z) p(z|\mathcal{D}_y) dz
$$

To calculate this, we will have to use the identity for a product of two Gaussians.

$$
\begin{align}
p(y|\mathcal{D}_y) &= \int \frac{1}{|w|} \mathcal{N}\left(z|\frac{y-b}{w}, \left(\frac{\sigma_{y|z}}{w}\right)^2\right) \mathcal{N}(z|\mu, \sigma^2) dz \\
&= \int \frac{1}{|w|} C^* \mathcal{N}\left(z|\mu^*, \left(\sigma^*\right)^2\right) dz \\
\end{align}
$$

#### Calculating the Predictive Posterior Distribution (PPD)

We can find these parameters from the identity for a product of two Gaussians.

$$
p(y|\mathcal{D}_y) = \int \frac{1}{|w|} C^* \mathcal{N}\left(z|\mu^*, \left(\sigma^*\right)^2\right) dz
$$

where

$\mu^* = \left(\sigma^* \right)^2 \left( \frac{\mu}{\sigma^2} + \frac{(y-b)/w}{\left(\frac{\sigma_{y|z}}{w}\right)^2} \right) = \left(\sigma^* \right)^2 \left( \frac{\mu}{\sigma^2} + \frac{(y-b)\cdot w}{\sigma_{y|z}^2} \right)$

$\left( \sigma^* \right)^2 = \frac{1}{\frac{1}{\sigma^2}+\frac{1}{\left( \frac{\sigma_{y|z}}{w}\right)^2}}= \frac{1}{\frac{1}{\sigma^2}+\frac{w^2}{\sigma_{y|z}^2}}$

$C^* = \frac{1}{\sqrt{2\pi \left( \sigma^2 + \frac{\sigma_{y|z}^2}{w^2} \right)}}\exp\left[ - \frac{\left(\mu - \frac{y-b}{w}\right)^2}{2\left( \sigma^2+\frac{\sigma_{y|z}^2}{w^2}\right)}\right]$

## In-class exercise

Integrate the PPD $p(y|\mathcal{D}_y)$.

### See you next class

Have fun!